In [16]:
import numpy as np
from numpy.linalg import inv
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from check_inverse import check_inverse_rank

def multi_class(X, y, Xnew, use_poly, lambda_ridge, order, mod):
    
    encoder = OneHotEncoder(sparse_output=False)
    ytr_onehot = encoder.fit_transform(y)
    
    print(f"ytr_onehot:")
    print(ytr_onehot)
    
    # poly
    if use_poly:
        poly = PolynomialFeatures(order)
        P = poly.fit_transform(X)
        P_new = poly.fit_transform(Xnew)
        
        print(f"P with shape {P.shape}:")
        print(P)
        print(f"Pnew with shape {P_new.shape}:")
        print(P_new)

        if (check_inverse_rank(P.T @ P)):
            w_poly = inv(P.T @ P) @ P.T @ ytr_onehot
            print(f"w (poly):")
            print(w_poly)
            
            # predict new input
            yts_pred = P_new @ w_poly
            print(f"pred_y (poly):\n {yts_pred}")
            
            # predict yts_pred_class
            yts_pred_class = np.argmax(yts_pred, axis=1) + 1
            print(f"yts_pred_class (poly):")
            print(yts_pred_class)
        else:
            if mod == "EE2213":
                L2_reg = np.identity(P.shape[1])
                w_poly_ridge = inv(P.T @ P + lambda_ridge * L2_reg) @ P.T @ ytr_onehot # EE2213 ver.
            else:
                L2_reg = np.identity(P.shape[0])
                w_poly_ridge = P.T @ inv(P @ P.T + lambda_ridge * L2_reg) @ ytr_onehot # EE2211 ver.
            print(f"w (poly ridge):")
            print(w_poly_ridge)
            
            # predict new input
            yts_pred = P_new @ w_poly_ridge
            print(f"pred_y (poly ridge): {yts_pred}")
            
            # predict yts_pred_class
            yts_pred_class = np.argmax(yts_pred, axis=1) + 1
            print(f"yts_pred_class (poly ridge):")
            print(yts_pred_class)
    # linear 
    else:
        w = inv(X.T @ X) @ X.T @ ytr_onehot
        print(f"w (linear): {w}")
        yts_pred = Xnew @ w
        
        print(f"yts_pred (linear):")
        print(yts_pred)
        
        yts_pred_class = np.argmax(yts_pred, axis=1)
        print(f"yts_pred_class (linear)")
        print(yts_pred_class)


In [ ]:
X = np.array([-1,0,0.5,0.3,0.8]).reshape(-1,1)
# X = np.column_stack([np.ones(X.shape[0]),X])
y = np.array([1,1,2,3,2]).reshape(-1,1)
X_test = np.array([[-0.1],[0.4]])
# X_test = np.column_stack([np.ones(X_test[0]),X_test])
use_poly = True
# set to 0 if lmbd = 0
lambda_ridge = 0
degree = 5
mod = "EE2211"
multi_class(X, y, X_test, use_poly, lambda_ridge, degree, mod)

ytr_onehot:
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
P with shape (5, 6):
[[ 1.      -1.       1.      -1.       1.      -1.     ]
 [ 1.       0.       0.       0.       0.       0.     ]
 [ 1.       0.5      0.25     0.125    0.0625   0.03125]
 [ 1.       0.3      0.09     0.027    0.0081   0.00243]
 [ 1.       0.8      0.64     0.512    0.4096   0.32768]]
Pnew with shape (2, 6):
[[ 1.000e+00 -1.000e-01  1.000e-02 -1.000e-03  1.000e-04 -1.000e-05]
 [ 1.000e+00  4.000e-01  1.600e-01  6.400e-02  2.560e-02  1.024e-02]]
matrix rank is : 5
matrix size is : (6, 6)
matrix is not invertible
w (poly ridge):
[[ 1.00000000e+00  4.61852778e-14 -2.55795385e-13]
 [-5.30313768e+00 -3.70234958e+00  9.00548727e+00]
 [ 5.21976232e+00  1.08728407e+01 -1.60926030e+01]
 [ 6.66624941e+00  9.46978846e+00 -1.61360379e+01]
 [-6.47651463e+00 -1.29098961e+01  1.93864107e+01]
 [-2.61986403e+00 -7.80449422e+00  1.04243583e+01]]
pred_y (poly ridge): [[ 1.57522369  0.46828063 -1.04350432]
 [-0.0